In [1]:
import sys
sys.path.insert(0, '/users/ankushroy/iris/src')
print(sys.path)

['/users/ankushroy/iris/src', '/users/ankushroy/iris/src/models', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python38.zip', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8/lib-dynload', '', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8/site-packages']


In [2]:
from typing import Any, Optional, Tuple

from einops import rearrange
import torch
import torch.nn as nn
import torch.nn.functional as F

import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf
import torch
import tqdm
import torch.nn as nn
from tqdm import tqdm

from dataset import Batch
from models.kv_caching import KeysValues
from models.slicer import Embedder, Head
from models.tokenizer import Tokenizer
from models.transformer import Transformer, TransformerConfig
from utils import init_weights, LossWithIntermediateLosses

In [3]:
# Define dataset
import torch
import sys

import h5py
from PIL import Image
#import matplotlib.pyplot as plt
from datetime import datetime, timedelta
def eventGeneration(start_time, obs_time = 3 ,lead_time = 6, time_interval = 30):
    # Generate event based on starting time point, return a list: [[t-4,...,t-1,t], [t+1,...,t+72]]
    # Get the start year, month, day, hour, minute
    year = int(start_time[0:4])
    month = int(start_time[4:6])
    day = int(start_time[6:8])
    hour = int(start_time[8:10])
    minute = int(start_time[10:12])
    #print(datetime(year=year, month=month, day=day, hour=hour, minute=minute))
    times = [(datetime(year, month, day, hour, minute) + timedelta(minutes=time_interval * (x+1))) for x in range(lead_time)]
    lead = [dt.strftime('%Y%m%d%H%M') for dt in times]
    times = [(datetime(year, month, day, hour, minute) - timedelta(minutes=time_interval * x)) for x in range(obs_time)]
    obs = [dt.strftime('%Y%m%d%H%M') for dt in times]
    obs.reverse()
    return lead, obs

from torch.utils.data import Dataset, DataLoader
import h5py
import numpy as np
from torchvision.transforms import ToTensor, Compose, CenterCrop
class radarDataset(Dataset):
    def __init__(self, root_dir, event_times, obs_number = 3, pred_number = 6, transform=None):
        # event_times is an array of starting time t(string)
        # transform is the preprocessing functions
        self.root_dir = root_dir
        self.transform = transform
        self.event_times = event_times
        self.obs_number = obs_number
        self.pred_number = pred_number
    def __len__(self):
        return len(self.event_times)
    def __getitem__(self, idx):
        start_time = str(self.event_times[idx])
        time_list_pre, time_list_obs = eventGeneration(start_time, self.obs_number, self.pred_number)
        output = []
        time_list = time_list_obs + time_list_pre
        #print(time_list)
        for time in time_list:
            year = time[0:4]
            month = time[4:6]
            #path = self.root_dir + year + '/' + month + '/' + 'RAD_NL25_RAC_MFBS_EM_5min_' + time + '_NL.h5'
            path = self.root_dir + year + '/' + month + '/' + 'RAD_NL25_RAP_5min_' + time + '.h5'
            image = np.array(h5py.File(path)['image1']['image_data'])
            #image = np.ma.masked_where(image == 65535, image)
            image = image[264:520,242:498]
            image[image == 65535] = 0
            image = image.astype('float32')
            image = image/100*12
            image = np.clip(image, 0, 128)
            image = image/40
            #image = 2*image-1 #normalize to [-1,1]
            output.append(image)
        output = torch.permute(torch.tensor(np.array(output)), (1, 2, 0))
        output = self.transform(np.array(output))
        return output
#root_dir = '/users/hbi/data/RAD_NL25_RAC_MFBS_EM_5min/'
#dataset = radarDataset(root_dir, ["200808031600"], transform = Compose([ToTensor(),CenterCrop(256)]))

In [4]:
# develop dataset
from torch.cuda.amp import autocast
#from torch.autograd import Variable
import pandas as pd
root_dir = '/home/hbi/RAD_NL25_RAP_5min/' 

df_train = pd.read_csv('/users/hbi/taming-transformers/training_Delfland08-14_20.csv', header = None)
event_times = df_train[0].to_list()
dataset_train = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))  

df_train_s = pd.read_csv('/users/hbi/taming-transformers/training_Delfland08-14.csv', header = None)
event_times = df_train_s[0].to_list()
dataset_train_del = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))  

df_test = pd.read_csv('/users/hbi/taming-transformers/testing_Delfland18-20.csv', header = None)
event_times = df_test[0].to_list()
dataset_test = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_vali = pd.read_csv('/users/hbi/taming-transformers/validation_Delfland15-17.csv', header = None)
event_times = df_vali[0].to_list()
dataset_vali = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_train_aa = pd.read_csv('/users/hbi/taming-transformers/training_Aa08-14.csv', header = None)
event_times = df_train_aa[0].to_list()
dataset_train_aa = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))  

df_train_dw = pd.read_csv('/users/hbi/taming-transformers/training_Dwar08-14.csv', header = None)
event_times = df_train_dw[0].to_list()
dataset_train_dw = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))    

df_train_re = pd.read_csv('/users/hbi/taming-transformers/training_Regge08-14.csv', header = None)
event_times = df_train_re[0].to_list()
dataset_train_re = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))   

data_list = [dataset_train_aa, dataset_train_dw, dataset_train_del, dataset_train_re]
train_aadedwre = torch.utils.data.ConcatDataset(data_list)

print(len(dataset_train), len(dataset_test), len(dataset_vali))
loaders = { 'train' :DataLoader(train_aadedwre, batch_size=1, shuffle=True, num_workers=8),
            'test' :DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=8), 
           'valid' :DataLoader(dataset_vali, batch_size=1, shuffle=False, num_workers=8),
          
          'train_aa5' :DataLoader(dataset_train_aa, batch_size=1, shuffle=False, num_workers=8),
          'train_dw5' :DataLoader(dataset_train_dw, batch_size=1, shuffle=False, num_workers=8),
          'train_del5' :DataLoader(dataset_train_del, batch_size=1, shuffle=True, num_workers=8),
          'train_re5' :DataLoader(dataset_train_re, batch_size=1, shuffle=False, num_workers=8),
          }

32183 3493 3560


In [6]:
from utils import configure_optimizer, EpisodeDirManager, set_seed
config =OmegaConf.load('/users/ankushroy/iris/config/trainer.yaml')
cfg=config
# Access the configuration and perform further operations
# For example, print the contents of the configuration
print(config)
print()

if config.common.seed is not None:
        set_seed(config.common.seed)

print(set_seed)

{'defaults': ['_self_', {'tokenizer': 'default'}, {'world_model': 'default'}, {'actor_critic': 'default'}, {'env': 'default'}, {'datasets': 'default'}], 'wandb': {'mode': 'online', 'project': 'iris', 'entity': None, 'name': None, 'group': None, 'tags': None, 'notes': None}, 'initialization': {'path_to_checkpoint': None, 'load_tokenizer': False, 'load_world_model': False, 'load_actor_critic': False}, 'common': {'epochs': 600, 'device': 'cuda:0', 'do_checkpoint': True, 'seed': 0, 'sequence_length': '${world_model.max_blocks}', 'resume': False}, 'collection': {'train': {'num_envs': 1, 'stop_after_epochs': 500, 'num_episodes_to_save': 10, 'config': {'epsilon': 0.01, 'should_sample': True, 'temperature': 1.0, 'num_steps': 200, 'burn_in': '${training.actor_critic.burn_in}'}}, 'test': {'num_envs': 8, 'num_episodes_to_save': '${collection.train.num_episodes_to_save}', 'config': {'epsilon': 0.0, 'should_sample': True, 'temperature': 0.5, 'num_episodes': 16, 'burn_in': '${training.actor_critic.b

In [7]:
import hydra
from omegaconf import OmegaConf

@hydra.main(config_path='/users/ankushroy/iris/config/', config_name='trainer.yaml')
def config_function(cfg):
    # Access the configuration
    config_file = OmegaConf.load(cfg)
  
    return config_file


cfg_worldmodel = config_function('/users/ankushroy/iris/config/world_model/default.yaml')
cfg_tokenizer = config_function('/users/ankushroy/iris/config/tokenizer/default.yaml')

In [8]:
device = torch.device(cfg.common.device)
print(device)


cuda:0


In [9]:
from collections import defaultdict
from functools import partial
from pathlib import Path
import shutil
import sys
import time
from typing import Any, Dict, Optional, Tuple

import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf
import torch
import tqdm
import torch.nn as nn
from tqdm import tqdm

#from agent import Agent
#from collector import Collector
#from make_reconstructions import make_reconstructions_from_batch

from utils import configure_optimizer, EpisodeDirManager, set_seed
from torch.utils.data import DataLoader
from datetime import datetime

In [11]:
from models.transformer import Transformer, TransformerConfig
block_mask = torch.cat([torch.ones(768), torch.zeros(1536)])
config = TransformerConfig(tokens_per_block=768, max_blocks=3, attention="causal", num_layers=6, num_heads=8, embed_dim=256, embed_pdrop=0.1, resid_pdrop=0.1, attn_pdrop=0.1)
transformer = Transformer(config)
head_observations = nn.Linear(256, 1024)
from models.world_model import WorldModel
tokenizer = instantiate(cfg_tokenizer)
#print(transformer)

Tokenizer : shape of latent is (1024, 8, 8).


In [13]:
from utils import configure_optimizer, EpisodeDirManager, set_seed
from models.world_model import WorldModel
tokenizer = instantiate(cfg_tokenizer)

#world_model = WorldModel(obs_vocab_size=tokenizer.vocab_size,config=instantiate(cfg_worldmodel))
#world_model.to(device)

#optimizer_trans = configure_optimizer(world_model, cfg.training.learning_rate, cfg.training.world_model.weight_decay)


Tokenizer : shape of latent is (1024, 8, 8).


In [14]:
ckpt_opt = torch.load('/space/ankushroy/Tokeniser_exp2_1024_embdim_10/vqvae_checkpoint_epoch350', map_location=device)

# Load model's state dict
model_state_dict = ckpt_opt['model_state_dict']
tokenizer.load_state_dict(model_state_dict)
tokenizer.to(device)



tokenizer

In [12]:
import torch
import time

# Clear CUDA cache
torch.cuda.empty_cache()

# Move the tokenizer to the device
tokenizer = tokenizer.to(device)

# Loop through the train loader
for i, images in enumerate(loaders['train']):
    if i < 0:
        continue
    if i >= 1:
        break
    
    # Preprocess the image
    image = images[0]
    image = image.unsqueeze(1)
    input_image = image.to(device)
    
    print("input image:", input_image.size())
    
    # Encode the image using the tokenizer
    encoding = tokenizer.encode(input_image)
    z, z_quantized, tokens = encoding.z, encoding.z_quantized, encoding.tokens
    
    print(z.size())
    print(z_quantized.size())
    print(tokens.size())


input image: torch.Size([9, 1, 256, 256])
torch.Size([9, 256, 16, 16])
torch.Size([9, 256, 16, 16])
torch.Size([9, 256])


In [20]:
device = torch.device(cfg.common.device)
print(device)
#from torch.autograd import Variable
name = ['train_aa5','train_dw5', 'train_del5','train_re5']
for i, images in enumerate(loaders['train']):
    a = images.to(device).unsqueeze(2).to(device)
    #print(a.shape)
    tokenizer = tokenizer.to(device)
    tokens = tokenizer.encode(input_image).tokens
    print(tokens.size())
    tokens_1 = torch.flatten(tokens).unsqueeze(0)
    print(tokens_1.size())
    torch.save([tokens_1], '/space/ankushroy/test1820iris/testset'+str(i)+'.pt')
    print(i, '/{}'.format(len(loaders['test'])), end='\r')


cuda:0
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torch.Size([9, 256])
torch.Size([1, 2304])
torc

In [15]:
import os
class latentDataset(Dataset):
    def __init__(self, root_dir, radarset):
        # event_times is an array of starting time t(string)
        # transform is the preprocessing functions
        self.root_dir = root_dir
        self.radarset = radarset
    def __len__(self):
        if self.radarset: return len(self.radarset)
        else: return 30000
    def __getitem__(self, idx):
        dir_file = self.root_dir + str(idx) + '.pt'
        if os.path.exists(dir_file):
            tokens_1 = torch.load(dir_file, map_location='cpu')
            return tokens_1
        else:
            print("File not found")
            return None


latent_test = latentDataset('/space/ankushroy/test1820iris/testset', dataset_test)


loaders_latent = { 'test_1' :DataLoader(latent_test, batch_size=1, shuffle=True, num_workers=0)}


In [16]:
import time
import matplotlib.pyplot as plt 
from pysteps.verification.detcatscores import det_cat_fct
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.spatialscores import intensity_scale
from pysteps.visualization import plot_precip_field
from einops import rearrange, repeat
threshold = 0.16
pcc_average = 0
pcc_average1 = 0
pcc_average2 = 0

# Assuming loaders_latent and tokens are already initialized
for i, latent in enumerate(loaders_latent['test_1']):
    print(type(latent))
    if i < 0:
        continue
    if i >= 1:
        break

    # Move the tokens to the device
    token = tokens_1[0].to(device)
    print(tokens_1.size())
    print(token.size())
    #z_q=tokenizer.embedding(token)
    # Rearrange the tensor using Einops
    z_q = rearrange(tokenizer.embedding(token), '(b h w) e -> b e h w', b=9, e=256, h=16, w=16).contiguous()
    
    # Decode using tokenizer
    reco = tokenizer.decode(z_q)
    print(reco.size())
    for t in range(9):
        a_r_display = reco[t,0,:,:].to('cpu').detach().numpy()*40
        a_r_display[a_r_display < threshold] = 0.0
        print(a_r_display.shape)
        plt.figure(figsize=(16, 4))

        plt.subplot(132)
        plot_precip_field(a_r_display, title="Reconstruction")
            
        plt.tight_layout()
        plt.show()



<class 'list'>


NameError: name 'tokens_1' is not defined

: 

In [ ]:
#torch.cuda.empty_cache()
import time
from pysteps.verification.detcatscores import det_cat_fct
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.spatialscores import intensity_scale
from pysteps.visualization import plot_precip_field
device = 'cuda:0'
pcc_average = 0
tokenizer=tokenizer.to(device)
for i, images in enumerate(loaders['test']):
    if i<0:continue
    if i>=2:break

    image = images[0]
    image = image.unsqueeze(1)
    image = image[:, :, :, :]
    input_image= image.to(device)  # batch x
        #print(input_image.size())
    a_r = tokenizer.encode_decode(input_image)
        #checkpoint = torch.load('/users/zboucher/iris/src/models/tokenizer/vqvae_epoch70', map_location = 'cpu')
        #vae.load_state_dict(checkpoint)
        #a_r2 = vae(a)

    for t in range(9):
        a_display = input_image[t,0,:,:].to('cpu').detach().numpy()*40
        a_r_display = a_r[t,0,:,:].to('cpu').detach().numpy()*40
        plt.figure(figsize=(16, 4))
        plt.subplot(131)
        plot_precip_field(a_display, title="Input")
        plt.subplot(132)
        plot_precip_field(a_r_display, title="Reconstruction")
            
        plt.tight_layout()
        plt.show()
print('pcc_average:', pcc_average/i)